In [ ]:
## TRAINING ONLY
#######################################################################################
import sys
sys.path.insert(0, '../') # go up 1 level to include the project root in the search path.

from models.MyResNet_Prefetcher import MyResNetPrefetcher
from utils.Reader import AdvancedReader
from utils.MyTimer import MyTimer

from sklearn.metrics import roc_curve, auc

import numpy as np
import tensorflow as tf

import pickle

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')

train_source = '/tmp/kaggle_dr_data/train_JF_BG_512/' 
val_source= '/tmp/kaggle_dr_data/test_JF_BG_512/' 
test_source = val_source 

train_csv_file = '/tmp/kaggle_dr_data/trainLabels.csv' # '/tmp/kaggle_dr_data/trainLabels.csv'
solution_csv_file = '/tmp/kaggle_dr_data/retinopathy_solution.csv' # '/tmp/kaggle_dr_data/retinopathy_solution.csv'

# Save results on GPFS01!
# You may wish to change this to your local folder.
RESULTS_DIR = '/gpfs01/berens/user/mayhan/Documents/MyPy/GitRepos/ttaug-DR-uncertainty/results/'

network_config = dict([('instance_shape', [512, 512, 3]),
                       ('num_classes', 5),
                       ('conv_depths', [1, 1, 1, 1]),
#                        ('num_filters', [[64, 64, 256], [128, 128, 512], [256, 256, 1024], [512, 512, 2048]]),
                       ('num_filters', [[64, 64, 128], [128, 128, 256], [256, 256, 512], [512, 512, 1024]]),
                       ('fc_depths', [512]),
                       ('lambda', 0.1),
                       ('lr', 0.005), 
                       ('momentum_max', 0.9),
                       ('decay_steps', 10000),
                       ('decay_rate', 0.8), 
                       ('data_aug', True),
                       ('data_aug_prob', 0.9),
                       ('max_iter', 500000),
                       ('oversampling_limit', 0.1),
                       ('batch_size', 23), # ResNet50: Max batch sizes allowed by BatchNorm and BatchReNorm are 14 and 8, respectively.
                       ('val_step', 5000),
                       ('resurrection_step', 25000), 
                       ('quick_dirty_val', False),
                       ('T', 0), # To be set later on during Test-time augmentation with various values, {4,8,16...}
                       ('dataset_buffer_size', 1000) # times minibatch size effectively
                      ])

model = MyResNetPrefetcher(network_config=network_config, name='MyResNet4DR_')
model.build()
model.initialize()
with MyTimer('Timer'):
    model.train(train_source=train_source, train_csv_file=train_csv_file,
                val_source=val_source, solution_csv_file=solution_csv_file)
model.finalize() 

diagnostics2save = model.diagnostics


result_file_name = RESULTS_DIR + model.descriptor + '_DIAG.pkl'
with open(result_file_name, 'wb') as filehandler:
    pickle.dump(diagnostics2save, filehandler, protocol=4)

# Clear some memory
del diagnostics2save

Total number of weight layers : 15
Residual stack depths : 1111 
Building the model graph...
Instance shape : [512, 512, 3]
Batch shape : [None, 512, 512, 3]
Num. of classes : 5
Data augmentation probability : 0.9
Biased-coin threshold : 0.1
conv1
	[7, 7], 64 /2
conv2/1
	[1, 1], 64 /1
	[3, 3], 64 /1
	[1, 1], 128 /1
conv3/1
	[1, 1], 128 /2
	[3, 3], 128 /1
	[1, 1], 256 /1
conv4/1
	[1, 1], 256 /2
	[3, 3], 256 /1
	[1, 1], 512 /1
conv5/1
	[1, 1], 512 /2
	[3, 3], 512 /1
	[1, 1], 1024 /1
fc1
	[2048, 512]
Regularization type : l1
logits
	[512, 5]


In [ ]:
## TRAINING CURVES and VALIDATION PERFORMANCE ACROSS TRAINING
#######################################################################################
with open(result_file_name, 'rb') as filehandler:
    diagnostics = pickle.load(filehandler)
    
    data1 = diagnostics['avg_losses']
    losses = diagnostics['avg_losses']
    tail_start = int(0.01*len(diagnostics['avg_losses']))
    print('Tail start : %g' % tail_start)
    data2 = losses[tail_start:]
    
    fig, ax1 = plt.subplots()    
    color = 'tab:red'
    ax1.set_xlabel('iteration')
    ax1.set_ylabel('Avg. minibatch loss', color=color)
    ax1.plot(range(0, len(diagnostics['avg_losses'])), data1, color=color, linestyle='--')
    ax1.tick_params(axis='y', labelcolor=color)
    
    ax2 = ax1.twinx()  # instantiate a second axis that shares the same x-axis
    color = 'tab:blue'
    ax2.set_ylabel('Avg. minibatch loss (zoomed in)', color=color)  # we already handled the x-label with ax1
    ax2.plot(range(tail_start, len(diagnostics['avg_losses'])), data2, color=color, linestyle=':')
    ax2.tick_params(axis='y', labelcolor=color)
    
    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.show()          
    
    
    # Validation performance across iterations
    plt.figure()
    
    x = np.multiply(model.network_config['val_step'], list(range(0, len(diagnostics['val_roc1']))))
    y = diagnostics['val_roc1']
    plt.plot(x, y, color='m', linestyle='--', label='onset 1')
    plt.plot(x, 0.889 * np.ones(shape=(len(y),1),dtype=np.float32), color='k', linestyle='--', label='Leibig et al., onset 1')
    
    x = np.multiply(model.network_config['val_step'], list(range(0, len(diagnostics['val_roc2']))))
    y = diagnostics['val_roc2']
    plt.plot(x, y, color='c', linestyle='-.', label='onset 2')
    plt.plot(x, 0.927 * np.ones(shape=(len(y),1),dtype=np.float32), color='k', linestyle='-.', label='Leibig et al., onset 2')
    
    plt.xlabel("iteration")
    plt.ylabel("ROC-AUC")
    #plt.title("ResNet50")
    plt.legend()
    plt.show()

# Clear some memory
del diagnostics

In [ ]:
## SINGLE PREDICTION RESULTS
#######################################################################################

model.restore()

print('=======================================================\nEvaluating the performance on TRAINING set')
# dr = AdvancedReader(source=train_source,
#                     csv_file='/gpfs01/berens/user/mayhan/kaggle_dr_data/trainLabels.csv', 
#                     mode = 'valtest' # valtest to read all from the training set
#                    )
labels_1hot_tr, predictions_1hot_tr, _, _, _, features_tr, logits_tr = model.inference(source=train_source,
                                                                                       csv_file=train_csv_file,
                                                                                       mode='valtest'
                                                                                      )

print('=======================================================\nEvaluating the performance on VALIDATION set')
# dr = AdvancedReader(source=test_source,
#                     csv_file='/gpfs01/berens/user/mayhan/kaggle_dr_data/retinopathy_solution.csv', 
#                     mode = 'val'
#                    )
labels_1hot_val, predictions_1hot_val, _, _, _, features_val, logits_val = model.inference(source=val_source,
                                                                                           csv_file=solution_csv_file,
                                                                                           mode='val'
                                                                                          )

print('=======================================================\nEvaluating the performance on TEST set')
# dr = AdvancedReader(source=test_source,
#                     csv_file='/gpfs01/berens/user/mayhan/kaggle_dr_data/retinopathy_solution.csv', 
#                     mode = 'test'
#                    )
labels_1hot_te, predictions_1hot_te, _, _, _, features_te, logits_te = model.inference(source=test_source, 
                                                                                       csv_file=solution_csv_file,
                                                                                       mode='test'
                                                                                      )

#### Now, save the results
result = {}
result['train_labels_1hot'] = labels_1hot_tr
result['val_labels_1hot'] = labels_1hot_val
result['test_labels_1hot'] = labels_1hot_te
result['train_pred_1hot'] = predictions_1hot_tr
result['val_pred_1hot'] = predictions_1hot_val
result['test_pred_1hot'] = predictions_1hot_te
result['train_features'] = features_tr
result['val_features'] = features_val
result['test_features'] = features_te
result['train_logits'] = logits_tr
result['val_logits'] = logits_val
result['test_logits'] = logits_te

result_file_name = RESULTS_DIR + model.descriptor + '_SINGpred.pkl'
with open(result_file_name, 'wb') as filehandler:
    pickle.dump(result, filehandler, protocol=4)

print('============\nComparing the performance with Christians VGG-like network')
# dr = AdvancedReader(source=test_source,
#                     csv_file='/gpfs01/berens/user/mayhan/kaggle_dr_data/retinopathy_solution.csv', 
#                     mode = 'test'
#                    )
labels_1hot_te, predictions_1hot_te, _, _, _, features_te, logits_te = model.inference(source=test_source, 
                                                                                       csv_file=solution_csv_file,
                                                                                       mode='valtest'
                                                                                      )
model.finalize()

# Clear some memory
del labels_1hot_tr, labels_1hot_val, labels_1hot_te
del predictions_1hot_tr, predictions_1hot_val, predictions_1hot_te
del features_tr, features_val, features_te
del logits_tr, logits_val, logits_te
del result

In [ ]:
## PLOTS FOR SINGLE PREDICTIONS
############################################################################################
from itertools import cycle
import scipy.stats as stats

def plot_roc_curve(labels, scores, linestyle, title, legend_prefix):
    
    # Compute ROC curve and ROC area for each class
    assert labels.shape[0] == scores.shape[0] and labels.shape[1] == scores.shape[1]

    n_classes = labels.shape[1]
    lw = 2
    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(labels[:, i], scores[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])   

    # Now plot the ROC-curves for classes
    colors = cycle(['red', 'green', 'blue', 'deepskyblue', 'blueviolet'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw, ls=linestyle, dash_joinstyle='bevel', dash_capstyle='butt', 
                 label=legend_prefix + ' Class {0} (area = {1:0.5f})' ''.format(i, roc_auc[i]))
    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)

def plot_roc_curves_for_all(result, title='Receiver Operating Characteristics'):
    """Inputs are in 1-hot or 1-vs-all format: Shape of [numOfExamples, numOfClasses]
    The function plots the ROC curves for each binary classification scenario.
    """
    
    labels_1hot_tr  = result['train_labels_1hot']
    labels_1hot_val = result['val_labels_1hot']
    labels_1hot_te = result['test_labels_1hot']
    #labels_1hot_valte = result['valtest_labels_1hot']
    predictions_1hot_tr = result['train_pred_1hot']
    predictions_1hot_val = result['val_pred_1hot']
    predictions_1hot_te = result['test_pred_1hot']
    #predictions_1hot_valte = result['valtest_pred_1hot']
    
    plot_roc_curve(labels=labels_1hot_tr, scores=predictions_1hot_tr, linestyle=':', title=title, legend_prefix='Train, ')
    plot_roc_curve(labels=labels_1hot_val, scores=predictions_1hot_val, linestyle='-.', title=title, legend_prefix='Val., ')
    plot_roc_curve(labels=labels_1hot_te, scores=predictions_1hot_te, linestyle='--', title=title, legend_prefix='Test, ')
    #plot_roc_curve(labels=labels_1hot_valte, scores=predictions_1hot_valte, linestyle='--', title=title, legend_prefix='ValTest, ')    
    
    leg = plt.legend(bbox_to_anchor=(1., 0.5, 0.625, 0.), loc='center right', ncol=1, mode="expand", shadow=True, fancybox=True)
    leg.get_frame().set_alpha(0.9)
    plt.show()

# Now, read the SINGLE PRED. results from file and plot
with open(result_file_name, 'rb') as filehandler:
    result = pickle.load(filehandler)
    
    # ROC curves for train,val and test data combined
    plt.figure()
    plot_roc_curves_for_all(result, '')

# Clear some memory
del result

In [ ]:
##################################################
### Test-time data augmentation for predictive uncertainty estimation
### via GENERATOR mechanism
##################################################

T_values = [4, 8, 16, 32, 64, 128]
for T in T_values:
    print('T = %g' % T)
    
    # model = MyResNet(network_config=network_config, name='MyResNet51_')
    model.restore()  # Restoring from meta-graph information does not permit new nodes in the graph 
    model.network_config['T'] = T
    
    print('=======================================================\nEvaluating the performance on TRAINING set')
    labels_1hot_tr, predictions_1hot_tr_ttaug, features_tr_ttaug, logits_tr_ttaug = model.inference_ttaug(source=train_source,
                                                                                                          csv_file=train_csv_file,
                                                                                                          mode='valtest'
                                                                                                         )
    
    print('=======================================================\nEvaluating the performance on VALIDATION set')
    labels_1hot_val, predictions_1hot_val_ttaug, features_val_ttaug, logits_val_ttaug = model.inference_ttaug(source=val_source,
                                                                                                        csv_file=solution_csv_file,
                                                                                                        mode='val'
                                                                                                       )
    
    print('=======================================================\nEvaluating the performance on TEST set')
    labels_1hot_te, predictions_1hot_te_ttaug, features_te_ttaug, logits_te_ttaug = model.inference_ttaug(source=test_source, 
                                                                                                          csv_file=solution_csv_file,
                                                                                                          mode='test'
                                                                                                         )
    model.finalize()
        
    #### Now, save the results
    result_ttaug = {}
    result_ttaug['train_labels_1hot'] = labels_1hot_tr
    result_ttaug['val_labels_1hot'] = labels_1hot_val
    result_ttaug['test_labels_1hot'] = labels_1hot_te
    result_ttaug['train_pred_1hot'] = predictions_1hot_tr_ttaug
    result_ttaug['val_pred_1hot'] = predictions_1hot_val_ttaug
    result_ttaug['test_pred_1hot'] = predictions_1hot_te_ttaug
    result_ttaug['train_logits'] = logits_tr_ttaug
    result_ttaug['val_logits'] = logits_val_ttaug
    result_ttaug['test_logits'] = logits_te_ttaug
    ## FEATURES TO BE SAVED SEPARATELY due to memory issues with T=128
    # This can be further improved by re-arranging the order of operations. However, for now, go with the flow!!!
#     result_ttaug['train_features'] = features_tr_ttaug
#     result_ttaug['val_features'] = features_val_ttaug
#     result_ttaug['test_features'] = features_te_ttaug
        
    result_file_name = RESULTS_DIR + model.descriptor + '_TTAUG_' + str(T) + '.pkl'
    with open(result_file_name, 'wb') as filehandler:
        pickle.dump(result_ttaug, filehandler, protocol=4)
    
    # Clear some memory
    del labels_1hot_tr, labels_1hot_val, labels_1hot_te
    del predictions_1hot_tr_ttaug, predictions_1hot_val_ttaug, predictions_1hot_te_ttaug
    del logits_tr_ttaug, logits_val_ttaug, logits_te_ttaug
    del result_ttaug
    
    # Now, save the FEATURES individually.
#     print('Saving FEAT VAL')
    result_file_name = RESULTS_DIR + model.descriptor + '_TTAUG_' + str(T) + '_FEATURES_val' + '.pkl'
    with open(result_file_name, 'wb') as filehandler:
        pickle.dump(features_val_ttaug, filehandler, protocol=4)
    del features_val_ttaug
    
#     print('Saving FEAT TR')
    result_file_name = RESULTS_DIR + model.descriptor + '_TTAUG_' + str(T) + '_FEATURES_train' + '.pkl'
    with open(result_file_name, 'wb') as filehandler:
        pickle.dump(features_tr_ttaug, filehandler, protocol=4)
    del features_tr_ttaug    
    
#     print('Saving FEAT TE')
    result_file_name = RESULTS_DIR + model.descriptor + '_TTAUG_' + str(T) + '_FEATURES_test' + '.pkl'
    with open(result_file_name, 'wb') as filehandler:
        pickle.dump(features_te_ttaug, filehandler, protocol=4)
    del features_te_ttaug